In [2]:
!pip install requests beautifulsoup4 pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

sheet1 = pd.read_excel(r"C:\Users\Croma Campus\Downloads\Address_mismatch.xlsx" , sheet_name="Sheet1")
sheet2 = pd.read_excel(r"C:\Users\Croma Campus\Downloads\Address_mismatch.xlsx" , sheet_name="Sheet2")


In [3]:
sheet1

,13179 5 BLACK MOUNTAIN RD SUITE 110
0,1518 24 THOMAS AVE
1,1640 206 CAMINO DEL RIO NORTH SUITE #107
2,6161 6161 SUITE 162
3,#1088 11160 RANCHO CARMEL DR SUITE 106
4,2 1/2 S 28TH ST
...,...
93,3748 1/2 CURTIS ST
94,3754 1/2 36TH ST
95,3820 1/2 GOLDFINCH ST
96,3854 1/2 36TH ST


In [8]:
# Normalize addresses
sheet1['Address_norm'] = sheet1['13179 5 BLACK MOUNTAIN RD SUITE 110'].str.lower().str.strip()
sheet2['Address_norm'] = sheet2['s1615 1/2 WLEWIS ST'].str.lower().str.strip()

sheet2

,s1615 1/2 WLEWIS ST,Address_norm
0,975R1/H NW OHRNBELND ST SUITE A2,975r1/h nw ohrnbelnd st suite a2
1,975 1/ 2 DIAmOND st,975 1/ 2 diamond st
2,97 0 1/2 TOURMALIN EST,97 0 1/2 tourmalin est
3,7W11 1/2 OLIEVR AVE SUIET 5,7w11 1/2 olievr ave suiet 5
4,747 1/2 SAN FEQRNNADO PL,747 1/2 san feqrnnado pl
...,...,...
93,115 1/2 CUSMHAN AVE,115 1/2 cusmhan ave
94,#188 11160 ARNCHO CARMEL DR SUITE 016,#188 11160 arncho carmel dr suite 016
95,2347 1/A ULRIC ST,2347 1/a ulric st
96,146 1/82 S 28TH ST,146 1/82 s 28th st


In [9]:
# Find common addresses
common_addresses = sheet1[sheet1['Address_norm'].isin(sheet2['Address_norm'])]

print(common_addresses["Address_norm"])


0        1518 24 thomas ave
2       6161 6161 suite 162
10          562 1/2 61st st
44      2326 1/2 julian ave
77    3282 1/2 imperial ave
93       3748 1/2 curtis st
96         3854 1/2 36th st
Name: Address_norm, dtype: object


In [6]:
common_addresses.to_csv("common_addresses.csv")

In [15]:
import requests
from bs4 import BeautifulSoup
import time


def get_county_osm(address):
    search_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": address,
        "format": "html"
    }

    response = requests.get(search_url, params=params)
    soup = BeautifulSoup(response.text, "html.parser")

    result = soup.find("span", class_="name")

    if not result:
        return None

    address_text = result.text.lower()

    # Extract county keyword
    words = address_text.split(',')
    for word in words:
        if "county" in word:
            return word.strip().title()

    return "County Not Found"


In [16]:
sheet1['County'] = sheet1['Address_norm'].apply(get_county_osm)

sheet1[['Address_norm', 'County']]


,Address_norm,County
0,1518 24 thomas ave,None
1,1640 206 camino del rio north suite #107,None
2,6161 6161 suite 162,None
3,#1088 11160 rancho carmel dr suite 106,None
4,2 1/2 s 28th st,None
...,...,...
93,3748 1/2 curtis st,None
94,3754 1/2 36th st,None
95,3820 1/2 goldfinch st,None
96,3854 1/2 36th st,None


In [17]:
sheet1["County"].unique()

array([None], dtype=object)

In [10]:
sheet1.to_csv("sheet1_with_county.csv")